In [24]:
import pandas as pd
import os

# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading in Data

In [25]:
df_train = pd.read_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/preprocessed_train_add.xlsx")
df_val = pd.read_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/preprocessed_val_add.xlsx")
df_test = pd.read_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/preprocessed_test_add.xlsx")

In [27]:
stocks = pd.read_csv("/content/drive/MyDrive/Masters Thesis/Colab notebook/Data scrapers/df_price_vix.csv") # For SPX500 + VIX
# stocks = pd.read_csv("/content/drive/MyDrive/Masters Thesis/Colab notebook/Data scrapers/spy_vix_combined.csv") # For SPY + VIX

First let us rewire the sentiment data so it is by traiding day to make sure furture sentiment does not leak backwards.

In [28]:
df_train.columns

Index(['Unnamed: 0', 'Sentiment', 'Date', 'X_Post', 'Reply_Count',
       'Repost_Count', 'Like_Count', 'View_Count', 'Follower_Count',
       'Verified_Status', 'Confidence', 'Sentiment_score', 'Like_Count_boxcox',
       'Repost_Count_boxcox', 'Reply_Count_boxcox', 'Engagement_Score',
       'Engagement_Impurity', 'Engagement_permutation', 'Engagement_SHAP',
       'Engagement_Equal', 'Engagement_Equal_Adjusted', 'SWIS_Equal',
       'Engagement_Score_Adjusted', 'SWIS_Ratio',
       'Engagement_Impurity_Adjusted', 'SWIS_Impurity',
       'Engagement_permutation_Adjusted', 'SWIS_perm',
       'Engagement_SHAP_Adjusted', 'SWIS_SHAP', 'Engagement_per_follower',
       'PCA_influnece', 'SWIS_metric', 'SWIS_PCA', 'PCA_influnece_abs'],
      dtype='object')

In [29]:
df_train[['Date']].max()

,0
Date,2023-10-30 23:47:11


In [30]:
df_train = df_train.sort_values('Date').reset_index(drop=True)
df_val = df_val.sort_values('Date').reset_index(drop=True)
df_test = df_test.sort_values('Date').reset_index(drop=True)

In [31]:
import pandas as pd
import pytz

def aggregate(df, swis_columns):
    """
    Aggregates SWIS variants into daily scores using tweets posted
    from T-1 at 16:00:01 ET through T at 16:00:00 ET.
    All weekend tweets (Fri 4PM – Mon 4PM ET) are assigned to Monday.
    """
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date_ET'] = df['Date'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

    def assign_trading_day(ts):
        cutoff_time = pd.to_datetime("16:00:00").time()

        # Weekend or Friday after 4PM: assign to next Monday
        if ts.weekday() in [5, 6] or (ts.weekday() == 4 and ts.time() > cutoff_time):
            days_ahead = (7 - ts.weekday()) % 7
            monday = (ts + pd.Timedelta(days=days_ahead)).date()
            return monday
        # Weekday after 4PM: assign to next day
        elif ts.time() > cutoff_time:
            return (ts + pd.Timedelta(days=1)).date()
        else:
            return ts.date()

    df['trading_day'] = df['Date_ET'].apply(assign_trading_day)

    agg_dict = {col: 'sum' for col in swis_columns}
    agg_dict['Sentiment_score'] = 'sum'

    daily = df.groupby('trading_day').agg(agg_dict).reset_index()
    daily = daily.sort_values('trading_day').reset_index(drop=True)

    return daily



In [32]:
df_train.columns

Index(['Unnamed: 0', 'Sentiment', 'Date', 'X_Post', 'Reply_Count',
       'Repost_Count', 'Like_Count', 'View_Count', 'Follower_Count',
       'Verified_Status', 'Confidence', 'Sentiment_score', 'Like_Count_boxcox',
       'Repost_Count_boxcox', 'Reply_Count_boxcox', 'Engagement_Score',
       'Engagement_Impurity', 'Engagement_permutation', 'Engagement_SHAP',
       'Engagement_Equal', 'Engagement_Equal_Adjusted', 'SWIS_Equal',
       'Engagement_Score_Adjusted', 'SWIS_Ratio',
       'Engagement_Impurity_Adjusted', 'SWIS_Impurity',
       'Engagement_permutation_Adjusted', 'SWIS_perm',
       'Engagement_SHAP_Adjusted', 'SWIS_SHAP', 'Engagement_per_follower',
       'PCA_influnece', 'SWIS_metric', 'SWIS_PCA', 'PCA_influnece_abs'],
      dtype='object')

In [33]:
swis_columns = ['SWIS_Equal', 'SWIS_Ratio', 'SWIS_Impurity', 'SWIS_perm', 'SWIS_SHAP', 'SWIS_metric', 'SWIS_PCA']

swis_train = aggregate(df_train, swis_columns)
swis_val   = aggregate(df_val, swis_columns)
swis_test  = aggregate(df_test, swis_columns)

In [34]:
swis_train.head(10)

,trading_day,SWIS_Equal,SWIS_Ratio,SWIS_Impurity,SWIS_perm,SWIS_SHAP,SWIS_metric,SWIS_PCA,Sentiment_score
0,2022-01-03,172.834665,248.101765,193.552471,183.690008,191.411167,0.019391,4.495752,5.564466
1,2022-01-04,478.950784,799.860947,575.092369,528.750556,553.653187,-0.025514,15.291032,9.896367
2,2022-01-05,-77.323765,-117.098227,-90.034397,-83.853929,-86.124607,-0.092959,-0.966468,-2.805749
3,2022-01-06,-342.626214,-666.061737,-442.495367,-394.155886,-416.204100,0.014154,-6.923462,-1.364992
4,2022-01-07,-166.268091,-197.601107,-180.145611,-173.152745,-170.973743,0.018934,-3.782034,1.967785
5,2022-01-10,-167.449695,-252.802607,-194.787185,-181.490600,-186.300198,-0.016572,-7.073186,-18.256184
6,2022-01-11,-308.202573,-558.821171,-385.441363,-348.065210,-365.299727,-0.074948,-6.943484,-4.990098
7,2022-01-12,99.489263,111.297715,101.790374,100.764828,102.950795,0.038107,6.872600,18.378030
8,2022-01-13,-219.014459,-450.619652,-289.603536,-255.497134,-272.235170,0.086773,-2.972173,2.746574
9,2022-01-14,98.387802,194.925884,127.397246,113.408299,120.809748,-0.079982,-0.190863,-6.973618


In [35]:
print(swis_train.shape)
print(swis_val.shape)
print(swis_test.shape)

(477, 9)
(155, 9)
(154, 9)


In [36]:
swis_columns = [
    'SWIS_Equal', 'SWIS_Ratio', 'SWIS_Impurity',
    'SWIS_perm', 'SWIS_SHAP', 'SWIS_metric', 'SWIS_PCA'
]

daily_scores = aggregate(df_train, swis_columns)

# 1. Create a complete list of business days from the first to last trading day
full_days = pd.date_range(
    start=df_train['trading_day'].min(),
    end=df_train['trading_day'].max(),
    freq='B'  # B = business day (Mon–Fri)
)

# 2. Compare with actual trading days present in your data
actual_days = pd.to_datetime(df_train['trading_day'])

# 3. Find missing trading days
missing_days = full_days.difference(actual_days)

# 4. Print the missing days
print("Missing trading days:")
print(missing_days)


Missing trading days:
DatetimeIndex([], dtype='datetime64[ns]', freq='B')


In [ ]:
swis_train = swis_train.rename(columns={'SWIS_Equal': 'IWSS_Equal', 'SWIS_Ratio': 'IWSS_Ratio', 'SWIS_Impurity': 'IWSS_Impurity', 'SWIS_perm': 'IWSS_perm', 'SWIS_SHAP': 'IWSS_SHAP',
                                        'SWIS_metric': 'IWSS_metric', 'SWIS_PCA': 'IWSS_PCA'})
swis_val = swis_val.rename(columns={'SWIS_Equal': 'IWSS_Equal', 'SWIS_Ratio': 'IWSS_Ratio', 'SWIS_Impurity': 'IWSS_Impurity', 'SWIS_perm': 'IWSS_perm', 'SWIS_SHAP': 'IWSS_SHAP',
                                    'SWIS_metric': 'IWSS_metric', 'SWIS_PCA': 'IWSS_PCA'})
swis_test = swis_test.rename(columns={'SWIS_Equal': 'IWSS_Equal', 'SWIS_Ratio': 'IWSS_Ratio', 'SWIS_Impurity': 'IWSS_Impurity', 'SWIS_perm': 'IWSS_perm', 'SWIS_SHAP': 'IWSS_SHAP',
                                      'SWIS_metric': 'IWSS_metric', 'SWIS_PCA': 'IWSS_PCA'})


In [ ]:
stocks.columns

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'VIX'], dtype='object')

Now, let's prepare the full price data

In [ ]:
stocks['Date'] = pd.to_datetime(stocks['Date'])

# Sort chronologically by Date
stocks = stocks.sort_values('Date').reset_index(drop=True)

# 3. Create 'trading_day' column (used for merging with tweet data)
stocks['trading_day'] = stocks['Date'].dt.date

# 4. Calculate current-day return (based on previous day's close)
stocks['return_t'] = stocks['Close'].pct_change()

# 5. Calculate next-day return (for use as prediction target)
stocks['return_t+1'] = stocks['Close'].pct_change().shift(-1)


In [ ]:
stocks.head()

,Date,Close,High,Low,Open,Volume,VIX,trading_day,return_t,return_t+1
0,2022-01-03,455.810425,455.944021,452.127392,454.465060,72668200,16.600000,2022-01-03,NaN,-0.000335
1,2022-01-04,455.657776,457.976400,453.778085,457.251231,71178700,16.910000,2022-01-04,-0.000335,-0.019202
2,2022-01-05,446.908142,456.068057,446.812721,455.285641,104538900,19.730000,2022-01-05,-0.019202,-0.000939
3,2022-01-06,446.488312,449.236289,444.093368,446.440616,86858900,19.610001,2022-01-06,-0.000939,-0.003954
4,2022-01-07,444.723083,447.690527,443.349095,446.497831,85111600,18.760000,2022-01-07,-0.003954,-0.001244


# Merging

In [ ]:
train_df = pd.merge(swis_train, stocks, on='trading_day', how='inner')
val_df = pd.merge(swis_val, stocks, on='trading_day', how='inner')
test_df = pd.merge(swis_test, stocks, on='trading_day', how='inner')

In [ ]:
train_df.head(10)

,trading_day,IWSS_Equal,IWSS_Ratio,IWSS_Impurity,IWSS_perm,IWSS_SHAP,IWSS_metric,IWSS_PCA,Sentiment_score,Date,Close,High,Low,Open,Volume,VIX,return_t,return_t+1
0,2022-01-03,172.834665,248.101765,193.552471,183.690008,191.411167,0.019391,4.495752,5.564466,2022-01-03,455.810425,455.944021,452.127392,454.465060,72668200,16.600000,NaN,-0.000335
1,2022-01-04,478.950784,799.860947,575.092369,528.750556,553.653187,-0.025514,15.291032,9.896367,2022-01-04,455.657776,457.976400,453.778085,457.251231,71178700,16.910000,-0.000335,-0.019202
2,2022-01-05,-77.323765,-117.098227,-90.034397,-83.853929,-86.124607,-0.092959,-0.966468,-2.805749,2022-01-05,446.908142,456.068057,446.812721,455.285641,104538900,19.730000,-0.019202,-0.000939
3,2022-01-06,-342.626214,-666.061737,-442.495367,-394.155886,-416.204100,0.014154,-6.923462,-1.364992,2022-01-06,446.488312,449.236289,444.093368,446.440616,86858900,19.610001,-0.000939,-0.003954
4,2022-01-07,-166.268091,-197.601107,-180.145611,-173.152745,-170.973743,0.018934,-3.782034,1.967785,2022-01-07,444.723083,447.690527,443.349095,446.497831,85111600,18.760000,-0.003954,-0.001244
5,2022-01-10,-167.449695,-252.802607,-194.787185,-181.490600,-186.300198,-0.016572,-7.073186,-18.256184,2022-01-10,444.169739,444.389176,435.668194,441.488559,119362000,19.400000,-0.001244,0.009108
6,2022-01-11,-308.202573,-558.821171,-385.441363,-348.065210,-365.299727,-0.074948,-6.943484,-4.990098,2022-01-11,448.215393,448.310815,440.868370,443.902613,74303100,18.410000,0.009108,0.002704
7,2022-01-12,99.489263,111.297715,101.790374,100.764828,102.950795,0.038107,6.872600,18.378030,2022-01-12,449.427155,451.507240,447.442520,449.971031,67605400,17.620001,0.002704,-0.013779
8,2022-01-13,-219.014459,-450.619652,-289.603536,-255.497134,-272.235170,0.086773,-2.972173,2.746574,2022-01-13,443.234619,451.201837,442.194591,450.543466,91173100,20.309999,-0.013779,0.000409
9,2022-01-14,98.387802,194.925884,127.397246,113.408299,120.809748,-0.079982,-0.190863,-6.973618,2022-01-14,443.415955,443.768988,438.816910,440.047781,95890900,19.190001,0.000409,-0.017710


In [ ]:
train_df.shape

(460, 18)

In [ ]:
train_df.to_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/combined_training_data_add_spyvix.xlsx", index=False)
val_df.to_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/combined_validation_data_add_spyvix.xlsx", index=False)
test_df.to_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/combined_test_data_add_spyvix.xlsx", index=False)

# Trying with new

In [ ]:
# Loading preprocessed data
df_train = pd.read_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/preprocessed_train.xlsx")
df_val = pd.read_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/preprocessed_val.xlsx")
df_test = pd.read_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/preprocessed_test.xlsx")

In [ ]:
# loading stock price data
stocks = pd.read_csv("/content/drive/MyDrive/Masters Thesis/Colab notebook/Data scrapers/df_price_vix.csv")
stocks.set_index("Date", inplace=True)
stocks.index = pd.to_datetime(stocks.index)
stocks = stocks.sort_index()

In [ ]:
# feature engineering

# --- Return & Trend-Based Features ---
stocks['ret_1d']   = stocks['Close'].pct_change()
stocks['ret_5d']   = stocks['Close'].pct_change(5)
stocks['ret_21d']  = stocks['Close'].pct_change(21)
stocks['reversal'] = -stocks['ret_1d']

# --- Volatility Features ---
stocks['vol_5d']   = stocks['ret_1d'].rolling(5).std()
stocks['vol_21d']  = stocks['ret_1d'].rolling(21).std()

# --- Liquidity Features ---
stocks['dollar_volume']  = stocks['Close'] * stocks['Volume']
stocks['amihud']         = stocks['ret_1d'].abs() / stocks['dollar_volume']
stocks['volume_zscore']  = (stocks['Volume'] - stocks['Volume'].rolling(21).mean()) / stocks['Volume'].rolling(21).std()

# --- Target Variable ---
stocks['Return_t+1'] = stocks['ret_1d'].shift(-1)

# === Select Base Features ===
base_features = stocks[[
    'ret_1d', 'ret_5d', 'ret_21d', 'reversal',
    'vol_5d', 'vol_21d',
    'dollar_volume', 'amihud', 'volume_zscore',
    'VIX',
    'Return_t+1'
]].dropna()

In [ ]:
base_features

,ret_1d,ret_5d,ret_21d,reversal,vol_5d,vol_21d,dollar_volume,amihud,volume_zscore,VIX,Return_t+1
Date,,,,,,,,,,,
2022-02-02,0.009423,0.055047,-0.043193,-0.009423,0.011492,0.011808,2.056767e+13,4.581226e-16,-0.498677,22.090000,-0.024391
2022-02-03,-0.024391,0.034885,-0.065943,0.024391,0.018931,0.012766,1.970956e+13,1.237526e-15,-0.593943,24.350000,0.005157
2022-02-04,0.005157,0.015497,-0.042559,-0.005157,0.016304,0.012322,2.118080e+13,2.434719e-16,-0.124073,23.219999,-0.003702
2022-02-07,-0.003702,-0.007016,-0.045183,0.003702,0.013805,0.012325,1.895996e+13,1.952385e-16,-0.833424,22.860001,0.008401
2022-02-08,0.008401,-0.005499,-0.033245,-0.008401,0.014048,0.012526,2.016435e+13,4.166366e-16,-0.515161,21.440001,0.014517
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23,0.007287,-0.016465,0.004263,-0.007287,0.015852,0.008261,2.146651e+13,3.394718e-16,-0.668279,16.780001,0.011043
2024-12-24,0.011043,-0.001747,0.011844,-0.011043,0.017055,0.008569,1.061670e+13,1.040131e-15,-1.975384,14.270000,-0.000406
2024-12-26,-0.000406,0.028172,0.008388,0.000406,0.005876,0.008553,1.753636e+13,2.313244e-17,-0.962232,14.730000,-0.011056


In [ ]:
# === Download Macroeconomic Indicators ===

import pandas as pd
from pandas_datareader import data as pdr

start = "2022-01-01"
end = "2025-01-01"

# --- Download each macroeconomic indicator individually ---

# 1. Consumer Price Index (CPI)
cpi = pdr.DataReader('CPIAUCSL', 'fred', start, end)
cpi.index = pd.to_datetime(cpi.index)
cpi = cpi.resample('D').ffill()
cpi.columns = ['CPI']

# 2. Unemployment Rate
unemployment = pdr.DataReader('UNRATE', 'fred', start, end)
unemployment.index = pd.to_datetime(unemployment.index)
unemployment = unemployment.resample('D').ffill()
unemployment.columns = ['Unemployment']

# 3. Federal Funds Rate
fed_funds = pdr.DataReader('FEDFUNDS', 'fred', start, end)
fed_funds.index = pd.to_datetime(fed_funds.index)
fed_funds = fed_funds.resample('D').ffill()
fed_funds.columns = ['FedFundsRate']

# 4. Term Spread (10-Year minus 2-Year Treasury)
term_spread = pdr.DataReader('T10Y2Y', 'fred', start, end)
term_spread.index = pd.to_datetime(term_spread.index)
term_spread = term_spread.resample('D').ffill()
term_spread.columns = ['TermSpread']

# 5. Industrial Production
industrial_prod = pdr.DataReader('INDPRO', 'fred', start, end)
industrial_prod.index = pd.to_datetime(industrial_prod.index)
industrial_prod = industrial_prod.resample('D').ffill()
industrial_prod.columns = ['IndustrialProduction']

# --- Merge all into one DataFrame ---
macro_combined = cpi \
    .join(unemployment, how='outer') \
    .join(fed_funds, how='outer') \
    .join(term_spread, how='outer') \
    .join(industrial_prod, how='outer')

# View result
print(macro_combined.head())



                CPI  Unemployment  FedFundsRate  TermSpread  \
DATE                                                          
2022-01-01  282.542           4.0          0.08         NaN   
2022-01-02  282.542           4.0          0.08         NaN   
2022-01-03  282.542           4.0          0.08        0.85   
2022-01-04  282.542           4.0          0.08        0.89   
2022-01-05  282.542           4.0          0.08        0.88   

            IndustrialProduction  
DATE                              
2022-01-01              101.2146  
2022-01-02              101.2146  
2022-01-03              101.2146  
2022-01-04              101.2146  
2022-01-05              101.2146  


In [ ]:
# Create macro regime flags
macro_combined['CPI_YoY'] = macro_combined['CPI'].pct_change(periods=365)
macro_combined['high_inflation'] = (macro_combined['CPI_YoY'] > 0.03).astype(int)
macro_combined['yield_curve_inverted'] = (macro_combined['TermSpread'] < 0).astype(int)
macro_combined['high_rates'] = (macro_combined['FedFundsRate'] > 3.0).astype(int)
macro_combined['high_unemployment'] = (macro_combined['Unemployment'] > 5.0).astype(int)


In [ ]:
# Select only macro flags to join
macro_flags = ['high_inflation', 'yield_curve_inverted', 'high_rates', 'high_unemployment']
macro_flag_df = macro_combined[macro_flags]

macro_flag_df.rename(columns={'DATE': 'Date'}, inplace=True)

<ipython-input-130-d1fddcfd53fa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macro_flag_df.rename(columns={'DATE': 'Date'}, inplace=True)


In [ ]:
model_data = base_features.join(macro_flag_df, how='left')

In [ ]:
model_data.head()

,ret_1d,ret_5d,ret_21d,reversal,vol_5d,vol_21d,dollar_volume,amihud,volume_zscore,VIX,Return_t+1,high_inflation,yield_curve_inverted,high_rates,high_unemployment
Date,,,,,,,,,,,,,,,
2022-02-02,0.009423,0.055047,-0.043193,-0.009423,0.011492,0.011808,2.056767e+13,4.581226e-16,-0.498677,22.090000,-0.024391,0,0,0,0
2022-02-03,-0.024391,0.034885,-0.065943,0.024391,0.018931,0.012766,1.970956e+13,1.237526e-15,-0.593943,24.350000,0.005157,0,0,0,0
2022-02-04,0.005157,0.015497,-0.042559,-0.005157,0.016304,0.012322,2.118080e+13,2.434719e-16,-0.124073,23.219999,-0.003702,0,0,0,0
2022-02-07,-0.003702,-0.007016,-0.045183,0.003702,0.013805,0.012325,1.895996e+13,1.952385e-16,-0.833424,22.860001,0.008401,0,0,0,0
2022-02-08,0.008401,-0.005499,-0.033245,-0.008401,0.014048,0.012526,2.016435e+13,4.166366e-16,-0.515161,21.440001,0.014517,0,0,0,0


In [ ]:
# === Create Interaction Terms ===
daily_features = ['ret_1d', 'ret_5d', 'reversal', 'vol_5d', 'vol_21d', 'amihud']

for f in daily_features:
    for m in macro_flags:
        model_data[f'{f}_x_{m}'] = model_data[f] * model_data[m]

In [ ]:
model_data[[f'{f}_x_{m}' for f in daily_features for m in macro_flags]].describe()


,ret_1d_x_high_inflation,ret_1d_x_yield_curve_inverted,ret_1d_x_high_rates,ret_1d_x_high_unemployment,ret_5d_x_high_inflation,ret_5d_x_yield_curve_inverted,ret_5d_x_high_rates,ret_5d_x_high_unemployment,reversal_x_high_inflation,reversal_x_yield_curve_inverted,...,vol_5d_x_high_rates,vol_5d_x_high_unemployment,vol_21d_x_high_inflation,vol_21d_x_yield_curve_inverted,vol_21d_x_high_rates,vol_21d_x_high_unemployment,amihud_x_high_inflation,amihud_x_yield_curve_inverted,amihud_x_high_rates,amihud_x_high_unemployment
count,731.000000,731.000000,731.000000,731.0,731.000000,731.000000,731.000000,731.0,731.000000,731.000000,...,731.000000,731.0,731.000000,731.000000,731.000000,731.0,7.310000e+02,7.310000e+02,7.310000e+02,731.0
mean,0.000440,0.000532,0.000716,0.0,0.002269,0.002694,0.003534,0.0,-0.000440,-0.000532,...,0.006477,0.0,0.003852,0.007037,0.006830,0.0,1.687410e-16,3.114330e-16,2.866017e-16,0.0
std,0.005532,0.008660,0.008144,0.0,0.012168,0.018299,0.016952,0.0,0.005532,0.008660,...,0.005224,0.0,0.004156,0.005235,0.004748,0.0,2.628663e-16,3.741637e-16,3.326790e-16,0.0
min,-0.020041,-0.043237,-0.029969,-0.0,-0.047589,-0.062784,-0.058454,-0.0,-0.022841,-0.055434,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,0.000000,-0.002607,-0.002116,0.0,0.000000,-0.002299,-0.002299,0.0,-0.000411,-0.003929,...,0.002773,0.0,0.000000,0.000000,0.004544,0.0,0.000000e+00,0.000000e+00,9.555653e-18,0.0
50%,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.006341,0.0,0.000000,0.007272,0.007198,0.0,0.000000e+00,1.796982e-16,1.640395e-16,0.0
75%,0.000411,0.003929,0.004234,0.0,0.004669,0.012567,0.012906,0.0,0.000000,0.002607,...,0.009244,0.0,0.007431,0.010517,0.009072,0.0,2.737918e-16,4.813915e-16,4.480418e-16,0.0
max,0.022841,0.055434,0.055434,-0.0,0.058525,0.063572,0.063572,-0.0,0.020041,0.043237,...,0.029036,0.0,0.012241,0.019924,0.019924,0.0,1.494563e-15,2.602445e-15,2.423598e-15,0.0


Zero values: Several features (like all _x_high_unemployment) have all 0s, suggesting the macro flag (high_unemployment) might be entirely false (0) across the dataset. You might want to confirm that.

Low means and stds: Values are small, as expected, since they’re products of returns/volatility with binary macro flags.

Sparse interactions: For many features, the 25%, 50%, and 75% percentiles are 0, meaning these macro conditions don’t happen often — which could impact model training.



In [ ]:
macro_flag_counts = model_data[macro_flags].astype(bool).sum()
print(macro_flag_counts.sort_values(ascending=False))


high_rates              564
yield_curve_inverted    540
high_inflation          354
high_unemployment         0
dtype: int64


In [ ]:
macro_flag_ratio = (model_data[macro_flags] != 0).mean().sort_values(ascending=False)
print((macro_flag_ratio * 100).round(2).astype(str) + '%')


high_rates              77.15%
yield_curve_inverted    73.87%
high_inflation          48.43%
high_unemployment         0.0%
dtype: object


It seems like high unemployment is always zero, so this interaction adds no information.

In [ ]:
# Drop interaction terms with high_unemployment
cols_to_drop = [col for col in model_data.columns if 'high_unemployment' in col]
model_data.drop(columns=cols_to_drop, inplace=True)


In [ ]:
# Drop the macro flag that's always zero
macro_flags_clean = [m for m in macro_flags if m != 'high_unemployment']

# Add VIX explicitly
extra_features = ['VIX']

# Rebuild the feature list
final_features = (
    daily_features +
    macro_flags_clean +
    [f'{f}_x_{m}' for f in daily_features for m in macro_flags_clean] +
    extra_features +
    ['Return_t+1']
)


# Subset and clean the dataset
final_dataset = model_data[final_features].dropna().copy()
final_dataset['trading_day'] = final_dataset.index


In [ ]:
final_dataset

,ret_1d,ret_5d,reversal,vol_5d,vol_21d,amihud,high_inflation,yield_curve_inverted,high_rates,ret_1d_x_high_inflation,...,vol_5d_x_high_rates,vol_21d_x_high_inflation,vol_21d_x_yield_curve_inverted,vol_21d_x_high_rates,amihud_x_high_inflation,amihud_x_yield_curve_inverted,amihud_x_high_rates,VIX,Return_t+1,trading_day
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-02,0.009423,0.055047,-0.009423,0.011492,0.011808,4.581226e-16,0,0,0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,22.090000,-0.024391,2022-02-02
2022-02-03,-0.024391,0.034885,0.024391,0.018931,0.012766,1.237526e-15,0,0,0,-0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,24.350000,0.005157,2022-02-03
2022-02-04,0.005157,0.015497,-0.005157,0.016304,0.012322,2.434719e-16,0,0,0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,23.219999,-0.003702,2022-02-04
2022-02-07,-0.003702,-0.007016,0.003702,0.013805,0.012325,1.952385e-16,0,0,0,-0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,22.860001,0.008401,2022-02-07
2022-02-08,0.008401,-0.005499,-0.008401,0.014048,0.012526,4.166366e-16,0,0,0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,21.440001,0.014517,2022-02-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23,0.007287,-0.016465,-0.007287,0.015852,0.008261,3.394718e-16,0,0,1,0.0,...,0.015852,0.0,0.0,0.008261,0.0,0.0,3.394718e-16,16.780001,0.011043,2024-12-23
2024-12-24,0.011043,-0.001747,-0.011043,0.017055,0.008569,1.040131e-15,0,0,1,0.0,...,0.017055,0.0,0.0,0.008569,0.0,0.0,1.040131e-15,14.270000,-0.000406,2024-12-24
2024-12-26,-0.000406,0.028172,0.000406,0.005876,0.008553,2.313244e-17,0,0,1,-0.0,...,0.005876,0.0,0.0,0.008553,0.0,0.0,2.313244e-17,14.730000,-0.011056,2024-12-26


In [ ]:
df_train = df_train.sort_values('Date').reset_index(drop=True)
df_val = df_val.sort_values('Date').reset_index(drop=True)
df_test = df_test.sort_values('Date').reset_index(drop=True)

In [ ]:
import pandas as pd
import pytz

def aggregate(df, swis_columns):
    """
    Aggregates SWIS variants into daily scores using tweets posted
    from T-1 at 16:00:01 ET through T at 16:00:00 ET.
    All weekend tweets (Fri 4PM – Mon 4PM ET) are assigned to Monday.
    """
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date_ET'] = df['Date'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

    def assign_trading_day(ts):
        cutoff_time = pd.to_datetime("16:00:00").time()

        # Weekend or Friday after 4PM: assign to next Monday
        if ts.weekday() in [5, 6] or (ts.weekday() == 4 and ts.time() > cutoff_time):
            days_ahead = (7 - ts.weekday()) % 7
            monday = (ts + pd.Timedelta(days=days_ahead)).date()
            return monday
        # Weekday after 4PM: assign to next day
        elif ts.time() > cutoff_time:
            return (ts + pd.Timedelta(days=1)).date()
        else:
            return ts.date()

    df['trading_day'] = df['Date_ET'].apply(assign_trading_day)

    agg_dict = {col: 'sum' for col in swis_columns}
    agg_dict['Sentiment_score'] = 'mean'

    daily = df.groupby('trading_day').agg(agg_dict).reset_index()
    daily = daily.sort_values('trading_day').reset_index(drop=True)

    return daily



In [ ]:
swis_train = swis_train.rename(columns={'SWIS_Equal': 'IWSS_Equal', 'SWIS_Ratio': 'IWSS_Ratio', 'SWIS_Impurity': 'IWSS_Impurity', 'SWIS_perm': 'IWSS_perm', 'SWIS_SHAP': 'IWSS_SHAP'})
swis_val = swis_val.rename(columns={'SWIS_Equal': 'IWSS_Equal', 'SWIS_Ratio': 'IWSS_Ratio', 'SWIS_Impurity': 'IWSS_Impurity', 'SWIS_perm': 'IWSS_perm', 'SWIS_SHAP': 'IWSS_SHAP'})
swis_test = swis_test.rename(columns={'SWIS_Equal': 'IWSS_Equal', 'SWIS_Ratio': 'IWSS_Ratio', 'SWIS_Impurity': 'IWSS_Impurity', 'SWIS_perm': 'IWSS_perm', 'SWIS_SHAP': 'IWSS_SHAP'})


In [ ]:
swis_columns = ['SWIS_Equal', 'SWIS_Ratio', 'SWIS_Impurity', 'SWIS_perm', 'SWIS_SHAP']

swis_train = aggregate(df_train, swis_columns)
swis_val   = aggregate(df_val, swis_columns)
swis_test  = aggregate(df_test, swis_columns)

In [ ]:
final_dataset['trading_day'] = pd.to_datetime(final_dataset['trading_day'])

# === Merge with Preprocessed Labels ===
swis_train['trading_day'] = pd.to_datetime(swis_train['trading_day'])
swis_val['trading_day'] = pd.to_datetime(swis_val['trading_day'])
swis_test['trading_day'] = pd.to_datetime(swis_test['trading_day'])

In [ ]:
merged_train = pd.merge(final_dataset, swis_train, on='trading_day', how='left')
merged_val   = pd.merge(final_dataset, swis_val, on='trading_day', how='left')
merged_test  = pd.merge(final_dataset, swis_test, on='trading_day', how='left')

In [ ]:
merged_train.columns

Index(['ret_1d', 'ret_5d', 'reversal', 'vol_5d', 'vol_21d', 'amihud',
       'high_inflation', 'yield_curve_inverted', 'high_rates',
       'ret_1d_x_high_inflation', 'ret_1d_x_yield_curve_inverted',
       'ret_1d_x_high_rates', 'ret_5d_x_high_inflation',
       'ret_5d_x_yield_curve_inverted', 'ret_5d_x_high_rates',
       'reversal_x_high_inflation', 'reversal_x_yield_curve_inverted',
       'reversal_x_high_rates', 'vol_5d_x_high_inflation',
       'vol_5d_x_yield_curve_inverted', 'vol_5d_x_high_rates',
       'vol_21d_x_high_inflation', 'vol_21d_x_yield_curve_inverted',
       'vol_21d_x_high_rates', 'amihud_x_high_inflation',
       'amihud_x_yield_curve_inverted', 'amihud_x_high_rates', 'VIX',
       'Return_t+1', 'trading_day', 'SWIS_Equal', 'SWIS_Ratio',
       'SWIS_Impurity', 'SWIS_perm', 'SWIS_SHAP', 'Sentiment_score'],
      dtype='object')

In [ ]:
merged_train.to_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/factors_train.xlsx", index=False)
merged_val.to_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/factors_val.xlsx", index=False)
merged_test.to_excel("/content/drive/MyDrive/Masters Thesis/Colab notebook/factors_test.xlsx", index=False)